In [1]:
pip install alpaca-trade-api yfinance pandas numpy xgboost scikit-learn joblib


     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB 1.3 MB/s eta 0:00:03
     ------ --------------------------------- 0.5/3.0 MB 5.7 MB/s eta 0:00:01
     ------------------------ --------------- 1.9/3.0 MB 14.9 MB/s eta 0:00:01
     ---------------------------------------  3.0/3.0 MB 19.2 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 17.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
INFO: pip is looking at multiple versions of yfinance to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/113.7 kB ? e

In [2]:
import yfinance as yf

symbols = ['AAPL', 'MSFT', 'TSLA', 'NVDA', 'GOOG']

for symbol in symbols:
    data = yf.download(tickers=symbol, period='7d', interval='5m')  # Last 7 days, 5-min bars
    data.to_csv(f'{symbol}_5min.csv')
    print(f"{symbol} saved!")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


AAPL saved!


[*********************100%***********************]  1 of 1 completed


MSFT saved!


...

TSLA saved!


[*********************100%***********************]  1 of 1 completed


NVDA saved!


[*********************100%***********************]  1 of 1 completed

GOOG saved!


In [29]:
import yfinance as yf
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import datetime

SYMBOLS = ['AAPL', 'MSFT', 'TSLA', 'NVDA', 'GOOG']
TRADE_QTY = 1
STOP_LOSS_PCT = 0.10
STARTING_CASH = 100000

def download_data(symbol):
    end = datetime.datetime.now()
    start = end - datetime.timedelta(days=7)
    print(f"Downloading {symbol} data...")
    df = yf.download(symbol, start=start, end=end, interval='5m', progress=False)
    df.reset_index(inplace=True)
    
    # Keep only necessary columns and add symbol column
    df = df[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']]
    df['symbol'] = symbol
    return df

def add_features(df):
    df = df.sort_values('Datetime').reset_index(drop=True)
    df['return'] = df['Close'].pct_change(fill_method=None)  # no fill to avoid warning
    df['volatility'] = df['return'].rolling(window=5).std()
    df['ma'] = df['Close'].rolling(window=5).mean()
    df['ma_diff'] = df['Close'] - df['ma']
    df['target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.dropna(inplace=True)
    return df

def train_model(df):
    df = add_features(df)
    X = df[['return', 'volatility', 'ma_diff']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    acc = model.score(X_test, y_test)
    print(f"Model trained. Accuracy: {acc*100:.2f}%")
    return model

def simulate_trading(data, model):
    cash = STARTING_CASH
    portfolio = {symbol: 0 for symbol in SYMBOLS}
    avg_price = {symbol: 0 for symbol in SYMBOLS}
    timestamps = sorted(data['Datetime'].unique())

    for i, ts in enumerate(timestamps):
        print(f"Time step {i+1}/{len(timestamps)}: {ts}")
        for symbol in SYMBOLS:
            df = data[(data['symbol'] == symbol) & (data['Datetime'] <= ts)].copy()
            if len(df) < 6:  # need enough data for rolling window
                continue
            df = add_features(df)
            features = df[['return', 'volatility', 'ma_diff']].iloc[[-1]]
            pred = model.predict(features)[0]
            price = df['Close'].iloc[-1]
            qty = portfolio[symbol]

            # Stop-loss check
            if qty > 0 and price < avg_price[symbol] * (1 - STOP_LOSS_PCT):
                cash += qty * price
                print(f"STOP-LOSS SELL {symbol} @ {price:.2f} qty: {qty}")
                portfolio[symbol] = 0
                avg_price[symbol] = 0
                continue

            # Buy signal
            if pred == 1 and qty == 0 and cash >= price * TRADE_QTY:
                cash -= price * TRADE_QTY
                portfolio[symbol] += TRADE_QTY
                avg_price[symbol] = price
                print(f"BUY {symbol} @ {price:.2f} qty: {TRADE_QTY}")

            # Sell signal
            elif pred == 0 and qty > 0:
                cash += qty * price
                print(f"SELL {symbol} @ {price:.2f} qty: {qty}")
                portfolio[symbol] = 0
                avg_price[symbol] = 0

            else:
                print(f"HOLD {symbol} @ {price:.2f} qty: {qty}")

        print(f"Cash: {cash:.2f}, Portfolio: {portfolio}\n")

    # Final portfolio
